# <b style="color: #151151">Lab 2 - Arend Wong (aw3146) - 2021-05-21<b>

<i style='color: blue;'>
Due Monday by 11:59pm Points 100 Submitting a website url or a file upload
Lab #2

1. Recode 2 different variables into new categories. They can both be continuous-ish or both be nominal-ish, or one of each. Tell me what you did and explain the variable(s).

2. Use one (or both) of your recoded variables to do a cross-tabulation. Explain your results.

3. Run a linear regression with 1 independent and 1 dependent variable; make all of the recodes necessary to make the model as easy to interpret as possible; and explain your results.

4. Plot two variables, either as a scatter plot or boxplot; add in trend/regression lines; and explain your results.
</i>


In [38]:
from __future__ import division
import pandas as pd
import numpy as np
import re, os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [8]:
# Read CSV into a dataframe
url = "https://raw.githubusercontent.com/the-pudding/data/master/pockets/measurements.csv"
df = pd.read_csv(url) # https://github.com/the-pudding/data/blob/master/pockets/measurements.csv

,brand,style,menWomen,name,fabric,price,maxHeightFront,minHeightFront,rivetHeightFront,maxWidthFront,minWidthFront,maxHeightBack,minHeightBack,maxWidthBack,minWidthBack,cutout
0,Arizona,skinny,women,Fave Super Skinny,"78% cotton, 20% polyester, 2% spandex",42.00,14.5,15.0,6.5,16.5,13.0,15.0,12.2,13.7,12.0,False
1,Arizona,straight,women,Perfect Boot,"78% cotton, 20% polyester, 2% spandex",42.00,14.5,14.0,6.5,16.0,12.5,15.5,12.2,13.0,11.2,False
2,Ralph Lauren,skinny,women,Modern Skinny Slimming Fit,"92% cotton, 7% poly, 1% elastane",89.50,13.0,13.5,6.5,14.5,12.0,15.5,13.0,13.5,12.5,False
3,Ralph Lauren,straight,women,Premier Straight Slimming Fit,"92% cotton, 7% poly, 1% elastane",89.50,13.0,13.5,6.5,14.5,12.0,14.5,13.0,13.0,12.2,False
4,Uniqlo,skinny,women,Skinny Fit,"87% cotton, 9% polyester, 4% spandex",39.90,13.0,13.0,5.5,14.0,11.5,14.0,12.0,14.2,11.7,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Wrangler,boot-cut,men,Regular Fit,"99% cotton, 1% spandex",48.00,28.0,15.5,9.0,13.0,12.0,16.7,13.5,14.5,14.0,False
76,Buckle Black,boot-cut,women,Fit No. 76 Bootcut Mid-Rise Curvy,"92% cotton, 6% polyester, 2% spandex",89.95,15.0,14.0,7.0,16.0,13.5,14.5,13.0,13.5,11.0,False
77,Buckle Black,skinny,women,Fit No. 53 Straight Mid-Rise Slim,"98% cotton, 2% elastane",92.95,14.5,13.5,6.0,16.0,13.0,14.0,12.5,14.0,11.5,False
78,Buckle Black,boot-cut,men,Fit No. 9 Bootleg,"92% cotton, 6% polyester, 2% elastane",94.95,21.0,18.5,8.5,18.5,14.5,16.0,14.0,13.0,12.5,False


In [45]:
# Create column for PRIMARY material

df_new = df.copy().assign(
        primary_fabric = df.fabric.apply(lambda s: s.split(', ')[0]),
        primary_fabric_pct = df.fabric.apply(lambda s: s.split(' ')[0]),
        primary_fabric_name = df.fabric.apply(lambda s: s.split(' ')[1].replace(',', ''))
         ).apply(lambda y: y.str.lower().str.strip().replace({'cottton':'cotton'}, regex=True) if str(y.dtype)=='object' else y)

df_new

,brand,style,menWomen,name,fabric,price,maxHeightFront,minHeightFront,rivetHeightFront,maxWidthFront,minWidthFront,maxHeightBack,minHeightBack,maxWidthBack,minWidthBack,cutout,primary_fabric,primary_fabric_pct,primary_fabric_name
0,arizona,skinny,women,fave super skinny,"78% cotton, 20% polyester, 2% spandex",42.00,14.5,15.0,6.5,16.5,13.0,15.0,12.2,13.7,12.0,False,78% cotton,78%,cotton
1,arizona,straight,women,perfect boot,"78% cotton, 20% polyester, 2% spandex",42.00,14.5,14.0,6.5,16.0,12.5,15.5,12.2,13.0,11.2,False,78% cotton,78%,cotton
2,ralph lauren,skinny,women,modern skinny slimming fit,"92% cotton, 7% poly, 1% elastane",89.50,13.0,13.5,6.5,14.5,12.0,15.5,13.0,13.5,12.5,False,92% cotton,92%,cotton
3,ralph lauren,straight,women,premier straight slimming fit,"92% cotton, 7% poly, 1% elastane",89.50,13.0,13.5,6.5,14.5,12.0,14.5,13.0,13.0,12.2,False,92% cotton,92%,cotton
4,uniqlo,skinny,women,skinny fit,"87% cotton, 9% polyester, 4% spandex",39.90,13.0,13.0,5.5,14.0,11.5,14.0,12.0,14.2,11.7,False,87% cotton,87%,cotton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,wrangler,boot-cut,men,regular fit,"99% cotton, 1% spandex",48.00,28.0,15.5,9.0,13.0,12.0,16.7,13.5,14.5,14.0,False,99% cotton,99%,cotton
76,buckle black,boot-cut,women,fit no. 76 bootcut mid-rise curvy,"92% cotton, 6% polyester, 2% spandex",89.95,15.0,14.0,7.0,16.0,13.5,14.5,13.0,13.5,11.0,False,92% cotton,92%,cotton
77,buckle black,skinny,women,fit no. 53 straight mid-rise slim,"98% cotton, 2% elastane",92.95,14.5,13.5,6.0,16.0,13.0,14.0,12.5,14.0,11.5,False,98% cotton,98%,cotton
78,buckle black,boot-cut,men,fit no. 9 bootleg,"92% cotton, 6% polyester, 2% elastane",94.95,21.0,18.5,8.5,18.5,14.5,16.0,14.0,13.0,12.5,False,92% cotton,92%,cotton


In [37]:
# Crosstabulation...

In [42]:
df_new.primary_fabric_name.value_counts()

cotton    78
rayon      1
Cotton     1
Name: primary_fabric_name, dtype: int64

In [44]:
# Linear regression
lm = smf.ols(formula = 'price~primary_fabric_name', data = df_new).fit()
print (lm.summary()) 

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     3.211
Date:                Sun, 23 May 2021   Prob (F-statistic):             0.0770
Time:                        17:50:01   Log-Likelihood:                -415.13
No. Observations:                  80   AIC:                             834.3
Df Residuals:                      78   BIC:                             839.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [ ]:
# p_value = 0.077 
# comparison factor: cotton
# There is a marginally significant relationship between primary fabric type and price of clothing item. The fabric rayon has, on average, a $79.24 (coefficient of 79.2405) higher price than does cotton (marginally significant with p=.077).


# for each $79.24 increase in 